# Set up ICU

In [1]:
%%capture
!pip install pyicu

In [2]:
import icu
def transliterator_from_rules(name, rules):
    fromrules = icu.Transliterator.createFromRules(name, rules)
    icu.Transliterator.registerInstance(fromrules)
    return icu.Transliterator.createInstance(name)

# Download data

In [3]:
_URL = "https://www3.smo.uhi.ac.uk/oduibhin/oideasra/lasid/lasid.zip"

In [4]:
%%capture
!wget {_URL}

In [5]:
%%capture
!unzip lasid.zip

In [6]:
file = open("mapdata.dat", "rb")

In [7]:
lines = file.readlines()

In [33]:
for line in lines:
  if b'\x09' in line:
    print(line)

b'066 R \t i h \xb2 n *\r\n'
b'088 k l \t i:   k l e i *\r\n'
b'018 b\xef \t \xaf u n *\r\n'
b'015 s \xd5\xef \t u: n *\r\n'
b'88S f \t i: *\r\n'
b'073 \xce t\xef \xad N t \t i   \xce h a r t *\r\n'
b'043 \xce f\xef \xb9: \t r b\xef i \xce k e: r d \xb2 \xce x\xef @ u: r \x11 \xb2 s t u \xfa \xad m *\r\n'


# LASID conversion

In [64]:
lasid_icu = """
\xb2 → ə ;
\xf9 → ʃ ;
#\x97 → α̩ ;
#\x97 → ɑ̜ ;
\x97 → α \u0323 ; # FIXME
\xaf → α ;
#\xaf → ɑ ;
\x9b → ø ;
\x94 → ö ;
\xef → ′ ;
\xf0 → ″ ;
\x9a → r̠ ;
\x8b → ï ;
\xce → ˈ ;
\x1f → ᵊ ;
\: → ː ;
I → ɪ ;
\xfa → ɣ ;
\xb9 → ɛ ;
\x8a → è ;
\x80 → ɸ ;
\@ → ʲ ;
\xb6 → ᾰ ;
\xe8 → ỹ ;
\xea → λ̃ ;
\xae → λ ;
\x11 → ʰ ;
\xb0 → ɔ ;
\x8d → ɔ̜ ;
\? → ʔ ;
\xde → k \u0323 ; # FIXME
\xe1 → g \u0323 ; # FIXME
\xe2 → e \u0323 ; # FIXME
\xdd → ö̤ ;
E → ᴇ ;
\xd4 → ᴇ̀ ;
\xac → ĕ ;
\x12 → ⁱ ;
\x83 → ɛ \u0300 ;
\xad → o̤ ;
\x99 → t̠ ; # toothache, 021
\x88 → ʈ ; # looks like t̜ : toothache, 033
\xfe → ŋ ;
\x87 → u̜ ; # finger-nails, 043
\xb7 → ă ;
\xcc → m̥ ; # awake, 001
\xf7 → n̥ ; # awake, 059
\xc7 → t \u0323 ; # probably t̞
\xa9 → ŏ ;
\xa6 → d̠ ; # wedge, 021
\_ → ǰ ; # crane, 021
\xf1 → ö̤̃ ; # dreaming, 078
\~ → ᵑ ; # dreaming, 078; maybe ⁿ̠ ?
\x1e → ᶾ ;
#\x9b → ø ;
\xb8 → λ \u0323 ; # FIXME
\xd1 → s \u0323 ; # FIXME
\x07 → ᵏ ;
#\x09 → ᵉ ;
\\\t → ᵉ ;
L → ʟ ;
N → ɴ ;
R → ʀ ;
"""

In [65]:
lasid = transliterator_from_rules('lasid_icu', lasid_icu)


In [66]:
test = b'066 R \t i h \xb2 n *\r\n'

In [67]:
lasid.transliterate(test.decode('ISO-8859-1').rstrip())

'066 ʀ ᵉ i h ə n *'